# Project description

-----------
We will be developing an agent-based model for the deer population in Michigan.  This will help us determine optimal hunting levels to maintain a stable deer population in various areas of Michigan.  One of the main concerns with deer in Michigan is that there are no longer enough hunters to control the population level.  Because of this, populations in many areas of Michigan, particularly southeast Michigan, have skyrocketed.  This can cause many issues for the people living in these areas since car accidents are more common.  There are also potential problems regarding the spread of chronic wasting disease (CWD) since the deer population is denser.  



## Data
---
We have found a lot of data regarding hunting of deer from the [MDNR website](https://www.michigan.gov/documents/dnr/2018_deer_harvest_survey_report_662078_7.pdf).  This will provide useful information on how exactly deer are hunted in Michigan and what attributes should put deer at risk.  This data is from 2018, so a lot of the information presented should still be accurate to the deer herd and hunting strategies now.


## Model description
---
The model will be an agent-based model that uses a class structure.  The primary class involved will be the deer class.  The methods that concern the deer class are:

1. Surviving (rate of survival)
2. Reproduction (seasonal)
3. Antler growth (seasonal)

Attributes for the deer would include:

1. Sex
2. Anterless or not (important for hunting)
3. Hunger
4. Age

Other potential classes that could be involved are:
1. Plants (food for deer)
2. Hunters
3. Cars

This model seems like it should be appropriate for the given data.  We could also compare it to simpler models such as the Lotka-Volterra model to see if we get the same results.  

Testing the model to see if it works may be difficult to do, since finding exact information regarding the total deer population levels has been difficult to do.  One possible solution to this is to just figure out how many hunters would be needed to maintain a deer population instead of trying to fit this model to real world data.

Below shows the importation of various python libaries that will be of use when simulating the model. The world of our model will be represented by a `numpy` array. The migration of each object, including deer, hunters, and cars, will be tracked by their respective positions on the numpy array.

In [257]:
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import display, clear_output

The class below is capable of instantiating various deer throughout the model. If a deer's location is not specified, a deer's location on the simulation board is chosen at random. This is done with respect to the given dimensions of the board. Similarly, if an age is not specified, then a deer's age will be given at random. Again, for the sex of the deer, if not specified, the chances of a deer being male or female are equal and chosen at random. As for pregnancy status, when a deer is instantiated, its `pregnant` attribute is set to `False`. This will be dealt with in a later method. The `antler` attribute indicates the presence or absence of antlers. Again, the deer will be initialized with not having antlers, as the complications of this phenomenon will be addressed in a method. The final attribute defines the `status` of the deer. Since this model is most-concerned with the dead population, it needs to be able to track the amount of deer that are still alive. This will allow the model to differentiate between a deer that is dead versus a deer that is alive.


The `move` method allows a deer to relocate (or not) during every timestep. The direction in which they move is chosen at random, however, they are bounded by the edges of the simulation board. 

In the `reproduce` method, female deer are capable of being impregnated under certain conditions during `'mating'` season. In order for mating to occur, a female deer must be in the same location as her male partner. Both of these deer need to be at least of age 1. This is typically the time when deer reach sexual maturity. Another feature of the `reproduce` method is that it is capable of producing a fawn. This can occur when the season is defined to be `'calving'`. If a female deer happens to be pregnant during this particular season, she gives birth to a fawn at her current location. Males are not too concerned with this method, as they are not capable of bearing a fawn.


A method that does concern the male deer is that of the `antler_status`. Male deer tend to grow their antlers around the same time that they reach sexual maturity. Once they reach this point, their antlers will grow seasonally. This progression begins in April and ends near the end of the calendar year where the antlers begin to deteriorate and fall off. Because hunting licenses are categorized in antler and antlerless sectors, this process of antler growth in male deer should present a much more accurate model.


In [31]:
def calc_distance(loc1, loc2):
    loc1 = np.array(loc1)
    loc2 = np.array(loc2)
    return np.sqrt(np.sum((loc1 - loc2)**2))

def show_animation(delay=0.01):
    fig = plt.gcf()
    time.sleep(delay)       # Sleep for half a second to slow down the animation
    clear_output(wait=True) # Clear output for dynamic display
    display(fig)            # Reset display
    fig.clear()             # Prevent overlapping and layered plots

class deer():
    '''
    This is the class that will model the deer population.  The deer's attributes include: sex, age, and
    and hunger.  Hunger will primarily be used to control the death of deer in the population.  If they
    reach a certain level, they will die.
    
    Methods:
        
    starve:
        If the deer's hunger reaches a certain amount (currently 20), the deer will die
        
    move:
        Picks a random available spot for the deer to move to
        
    draw:
        Plots the deer's position on a scatter plot.
    '''
    def __init__(self, dim_x, dim_y, loc = None, age = None, sex = None, age_cdf = [0.5, 0.75, 0.85, 0.95, 1]):
        
        if loc == None:
            self.loc = [np.random.randint(dim_x), np.random.randint(dim_y)]
        else:
            self.loc = loc
        # Finds the age of the deer.  If the age is not given, it uses the cdf of the age
        # distribution to pick a random age for the deer
        if age == None:
            rand_number = np.random.random()
            for i in range(len(age_cdf)):
                if rand_number > age_cdf[i]:
                    continue
                else:
                    self.age = i
                    break
        else:
            self.age = age
        
        # Finds the sex of the deer.  If the sex is not given, randomly picks between male
        # and female
        if sex == None:
            self.sex = np.random.choice(['m','f'])
        else:
            self.sex = sex
            
        #Sets reproduction status
        self.pregnant = False
        

        # Sets a hunger level for the deer
        #self.hunger = 0
        self.status = 'alive'
        
        #Sets antler status
        self.antler = False

    def survive(self):
        if self.age == 0:
            year_survival = 0.6
        else:
            year_survival = 0.9
        daily_survival = year_survival**(1/365)
        if np.random.random() > daily_survival:
            self.status = 'dead'
            
    def move(self, dim_x, dim_y):
        choices = ['stay']
        
        if self.loc[0] > 0:
            choices.append('left')
        if self.loc[0] < dim_x - 1:
            choices.append('right')
        if self.loc[1] > 0:
            choices.append('down')
        if self.loc[1] < dim_y - 1:
            choices.append('up')
        
        direction = np.random.choice(choices)
        
        if direction == 'up':
            self.loc[1] += 1
        elif direction == 'down':
            self.loc[1] -= 1
        elif direction == 'right':
            self.loc[0] += 1
        elif direction == 'left':
            self.loc[0] -= 1
            
    def draw(self):
        if self.status == 'alive':
            plt.scatter(*self.loc, marker = '*', color = 'brown', alpha = 0.25)
        else:
            pass
    
    
    # Creates an instance at current location if conditions are met 
    def reproduce(self, season, deer_):
        # Females must be at least 1 year of age
        if self.sex == 'f' and self.age > 1:
            # Become potentiall pregnant during mating season
            if season == 'mating':
                for d in deer_:
                    if d.sex == 'm' and d.loc == self.loc and d.age > 1:
                        self.pregnant = True
            # If previously pregnant, a fawn is spawned at mother's location
            if season == 'calving' and self.pregnant = True :
                self.pregnant = False
                newborn = deer(x_dim, y_dim, age = 0)
                return self, newborn
        return self, None
    
    
    # Anterless deer are considered to have less than three inches of antler length (April to January)
    def antler_status(self, time):
        # Males need to be older that 1
        t = time % 365
        if self.sex == 'm' and self.age > 1:
            # Antlered begining in April
            if t > 91:
                self.antler = True
            # Antlerless for rest of year
            else:
            self.antler = False
        # No females are antlered for our purposes 
        else:
            self.antler = False


In [3]:
class hunter():
    def __init__(self, dim_x, dim_y):
        self.loc = [np.random.randint(dim_x), np.random.randint(dim_y)]
        self.hunt_number = 0
        
        limit_cdf = [0.75, 0.95, 1]
        random_number = np.random.random()
        for i in range(3):
            if random_number > limit_cdf[i]:
                continue
            else:
                self.hunt_limit = i + 1
                break
        
        # Selects the type of hunter with their respective probabilities 
        self.antler = np.random.choice([True, False], p = [0.58, 0.42])                    
    
    def move(self, dim_x, dim_y):
        choices = ['stay']
        
        if self.loc[0] > 0:
            choices.append('left')
        if self.loc[0] < dim_x - 1:
            choices.append('right')
        if self.loc[1] > 0:
            choices.append('down')
        if self.loc[1] < dim_y - 1:
            choices.append('up')
        
        direction = np.random.choice(choices)
        
        if direction == 'up':
            self.loc[1] += 1
        elif direction == 'down':
            self.loc[1] -= 1
        elif direction == 'right':
            self.loc[0] += 1
        elif direction == 'left':
            self.loc[0] -= 1
            
    def hunt(self, deer_, hunter_range, season):
        if self.hunt_number < self.
        for d in deer_:
            if calc_distance(self.loc, d.loc) < hunter_range and self.antler == d.antler and season == True:
                d.status = 'dead'
                self.hunt_number += 1
                break
            else:
                continue
        return deer_
    
    
    def draw(self):
        plt.scatter(*self.loc, marker = '^', color = 'orange')

In [2]:
class car():
    '''
    This class will introduce cars into the model. Cars will occupy only half of the dimensions of the model to
    simulate a somewhat urban setting in addition to an entirely natural environment. Simply, if a car is in the 
    same location as a deer at a given time step, the deer will be eliminated. 
    '''
    def __init__(self, dim_x, dim_y):
        
        dim_x = dim_x/2
        dim_y = dim_y/2
        
        self.dim_x = dim_x/2
        self.dim_y = dim_y/2
        
        self.loc = [np.random.randint(dim_x), np.random.randint(dim_y)]
        
    def move(self):
        
        dim_x = self.dim_x
        dim_y = self.dim_y
        
        choices = ['stay']
        
        if self.loc[0] > 0:
            choices.append('left')
        if self.loc[0] < dim_x - 1:
            choices.append('right')
        if self.loc[1] > 0:
            choices.append('down')
        if self.loc[1] < dim_y - 1:
            choices.append('up')
        
        direction = np.random.choice(choices)
        
        if direction == 'up':
            self.loc[1] += 1
        elif direction == 'down':
            self.loc[1] -= 1
        elif direction == 'right':
            self.loc[0] += 1
        elif direction == 'left':
            self.loc[0] -= 1        
    
    def hit(self, deer_):
        for d in deer_:
            if self.loc == d.loc:
                d.status = 'dead'
            else:
                continue
        return deer_
    
    def draw(self):
        plt.scatter(*self.loc, marker = '+', color = 'red')

In [282]:
class world():
    '''
    This class will be for the world in the model.  It will include deer, vegetation, hunters, etc. 
    As attributes and will have methods to advance the stage of the board.
    '''
    
    def __init__(self, dim_x, dim_y, n_deer, n_hunters, n_cars, time = 0):
        # initializes the board as a numpy array with only zeros
        self.board = np.zeros((dim_y, dim_x))
        self.dim_x = dim_x
        self.dim_y = dim_y
        
        # creates the initial deer population
        deer_ = []
        for i in range(n_deer):
            deer_.append(deer(dim_x, dim_y))
        
        # creates hunter population
        hunters = []
        for i in range(n_hunters):
            hunters.append(hunter(dim_x, dim_y))
            
        # creates car population
        cars = []
        for i in range(n_cars):
            cars.append(car(dim_x, dim_y))
            
                    
        # creates attributes for the world class for deer, hunters, and cars
        # time attribute is just the number of days from the start
        # can assume that the model will always start on January 1st
        # and we dont take leap years into account.
        self.deer_ = deer_
        self.all_deer = deer_
        self.hunters = hunters
        self.cars = cars
        self.time = time
        
    def calc_hunt_season(self):
        # calcs the status of the hunting season.  It seems that the hunting season generally
        # starts in september and continues until the end of the year
        
        t = self.time % 365
        if t > 243:
            self.hunt_season = True
        else:
            self.hunt_season = False  
        
    def calc_reproduction_season(self):
        # calcs the status of the deer reproduciton season.  Generally the deer will be mating
        # in october-december and will actually give birth to calves in may-july
        t = self.time % 365
        
        if t > 273:
            self.reproduction_season = 'mating'
        elif (t > 120) and (t < 181):
            self.reproduction_season = 'calving'
        else:
            self.reproduction_season = None
        
    def draw(self):
        # plots all of the populations on the board
        plt.imshow(self.board, cmap = 'Greys')

        for d in self.deer_:
            d.draw()
        for h in self.hunters:
            h.draw()
        for c in self.cars:
            c.draw()
        
    def advance(self):
        # advances the state of the board by one day
        self.calc_hunt_season()
        self.calc_reproduction_season()
        
        for d in self.deer_:
            if d.status == 'dead':
                self.deer_.remove(d)
                continue
                
            d.move(self.dim_x, self.dim_y)
                
            d.survive()
            
            d.reproduce()
            
            d.antler_status()
        
        
        
        if self.calc_hunt_season == True:
            for h in self.hunters:
        
                h.move(self.dim_x, self.dim_y)
            
                h.hunt(deer_, hunter_range)
            
        
        for c in self.cars:
            
            c.move()
            
            c.hit(deer_)
        
        
        
        self.time += 1
        
        if self.time % 365 == 0:
            for d in self.deer_:
                d.age += 1
        
                    
    def calc_deer_pop(self):
        # finds the number of living deer in the population
        N = 0
        for d in self.deer_:
            if d.status == 'alive':
                N += 1
            
        return N
            

[1, 2, 3, 2, 4, 5]
[1, 2, 3, 2, 4, 5]
[1, 3, 2, 4, 5]
[1, 3, 4, 5]
[1, 3, 4, 5]


## Results

## Conclusions

## Roles

For now, we are both going to be doing research to look for more data regarding the deer herd and potentially look at information from other nearby states.  Noah will start a simple definition of the deer class. 